In [1]:
from pathlib import Path
from sys import path

from polyfactory.factories.sqlalchemy_factory import SQLAlchemyFactory

path.append(str(Path("..").absolute()))
from api.db.developers import Developer

In [ ]:
from geoalchemy2 import Geometry
from polyfactory import Ignore, Use


class Factory(SQLAlchemyFactory[Developer]):
    id = Ignore()
    name = Use(SQLAlchemyFactory.__random__.choice, ["James", "Mary", "John", "Patricia", "Robert"])
    department = Use(SQLAlchemyFactory.__random__.choice, ["backend", "frontend", "ios", "android"])

    @classmethod
    def get_sqlalchemy_types(cls):
        return {**super().get_sqlalchemy_types(), Geometry: cls.get_geometry_type}

    @classmethod
    def get_geometry_type(cls):
        lat, lot = cls.__faker__.latlng()
        return f"SRID=4269;POINT({lot} {lat})"


f = Factory()

In [13]:
f.build()

Developer(name='Patricia', last_known_ip='195.14.74.175', department='backend', is_available=True, geolocation='SRID=4269;POINT(75.239787 -40.0917505)')

In [ ]:
from api.conf import session_maker

async with session_maker() as session:
    for _ in range(20):
        obj = f.build()
        session.add(obj)

    await session.commit()

In [25]:
class RandomFactory(Factory):
    name = Use(lambda: SQLAlchemyFactory.__faker__.name())
    department = Use(lambda: SQLAlchemyFactory.__faker__.job())


random_factory = RandomFactory()

In [33]:
random_factory.build()

Developer(name='Mackenzie Daniels', last_known_ip='118.247.114.71', department='Clinical psychologist', is_available=True, geolocation='SRID=4269;POINT(152.774864 -10.9052985)')

In [34]:
async with session_maker() as session:
    for _ in range(10_000):
        obj = random_factory.build()
        session.add(obj)

    await session.commit()